<div class="alert alert-block alert-info">
     <b>Import Modules Needed</b> <i>(need pandas >= 1.2 and python >= 3.9.0 for all the data cleaning techniques to work)</i>
</div>

In [ ]:
import pandas as  pd
import os
import sweetviz as sv
import numpy as np
import dtale
import cudf
import dask_cudf
from pandasql import sqldf

#!pip install git+https://github.com/innovationOUtside/fstring-magic.git

In [ ]:
%load_ext fstring_magic

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [ ]:
# WINDOWS - get IP address of running Ubuntu instance
# Run an admin elevated powershell instance (not ubuntu VM)
# netsh interface portproxy add v4tov4 listenport=40000 listenaddress=0.0.0.0 connectport=40000 connectaddress=172.21.240.230
# This will open up a port that the dtale package between local windows machine and the Ubuntu VM need to display correctly
!ifconfig
# netsh interface portproxy add v4tov4 listenport=40000 listenaddress=0.0.0.0 connectport=40000 connectaddress=172.21.240.230


In [ ]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-block alert-info">
    <b>Find CSV Files in working dir</b>
</div>

In [ ]:
csvStore = os.path.join(os.getcwd(),'__csvStore' )
csvlist = []
for file in enumerate(os.listdir(csvStore)):
    if file[1].endswith(".csv"):
        csvlist.append(os.path.join(str(csvStore), file[1]))
        print(os.path.join(str(csvStore), file[1]))
        

<div class="alert alert-block alert-info">
    <b>Import the raw data</b>
</div>

I learned through manual trial and error many different data issues when trying to import the csv <br>
The main issue was with datetimes, and columns recognized as containing mixed data types. <br>
Some columns can be converted during import but other must be coerced after <br>



In [ ]:
csvfile = 'orig_data2.csv'
list(enumerate(csvlist))
csvlist[4]

In [ ]:
df = pd.read_csv(csvlist[4], na_values = ['IGNORED', '-', 'UNKNOWN', 'UK - OTHER UNKNOWN', 'UK - UNKNOWN'], dtype = {'ceas_ascode': object, 'M1322_NBR_PRSULC_STG1':object})



In [ ]:
%%fstring
__There are {len(df.columns)} columns, in the df__

<div class="alert alert-block alert-success">
    <b>Create SweetViz EDA Report</b>
</div>

In [ ]:
#analyzing the dataset
hospice_death_report = sv.analyze(df, pairwise_analysis = 'off')
#display the report
hospice_death_report.show_html('cleaned_hospice_dead.html')


<hr style="height:2px;border-width:0;color:black;background-color:black">

# Part 1 Cleanup of full dataset

<div class="alert alert-danger" role="alert">
    <b>Manually Clean up Data (remove cols) based on SweetViz report</b>
</div>

In [ ]:
df_clean = df.drop(df.columns[np.r_[3, 5, 14, 15, 36, 47:70, 87:94, 100:104, 105:107, 122:129, 136, 138:149, 151, 168, 178:184, 192:194, 195:198, 201, 204:211, 212:218, 219, 221:223, 224:229, 230:232, 240:261, 261:266]], axis = 1)
                                    
#df_clean.columns


# REMOVE COLS WITH ONLY NAN
df = df.dropna(axis=1, how='all')




In [ ]:
%%fstring
__There are {len(df.columns)} columns remaining if we simply eliminate columns with 100% null values, or {len(df_clean.columns)} based on my manual removal process__

<div class="alert alert-danger" role="alert">
    <b>Compare manual elimination of empty columns via SweetViz to pandas automatic removal</b>
</div>

In [ ]:
(df.columns).difference(df_clean.columns)

In [ ]:
df = df.drop((df.columns).difference(df_clean.columns), axis = 1)
len(df.columns)
len(df.columns) == len(df_clean.columns)

In [ ]:
%%fstring
__We can remove these columns even though they are not strictly NAN. They are majority NAN and are not useful to the analysis__

<div class="alert alert-block alert-info">
    <b>Find Columns with Dates and convert them to datetime type</b>
</div>

In [ ]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OFEPISODE', case=False))] 
df[cols_w_dates] = df[cols_w_dates].apply(pd.to_datetime)
cols_w_dates

<div class="alert alert-block alert-info">
    <b>Find Columns that contain mixed data types to investigate for possible data quality issues</b>
</div>


In [ ]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

In [ ]:
# If Run in Linux pd.read_csv was detecting mixed datatype differently than windows

In [ ]:
# # Coerce bad columns to int dtype
# for col in ['M1311_NSTG_CVRG_SOCROC_E2','M1322_NBR_PRSULC_STG1', 'M2200_THER_NEED_NBR']:
#   df[col] = pd.to_numeric(df[col], errors='coerce')
         
for col in ['M1322_NBR_PRSULC_STG1']:
   df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# Descriptive IDs
df.columns[:26]
colsMeta = df.columns[:26]
# OASIS Factors
df.columns[26:]
colsOASIS = df.columns[26:]

<div class="alert alert-info" role="alert">
    <b>Percentage of Null Values by Column to see if there are errors or we can eliminate unhelpful columns</b>
</div>


In [ ]:
(df.loc[:,colsOASIS].isnull().sum()/len(df.loc[:,colsOASIS])).sort_values(ascending=False)

> Since we previously eliminated completely empty columns we should not expect to see empty columns here. We see the columns ranked by degree of missing values <BR>
  Prime candidates for elimination are columns with 90% or greater nulls. We need both training and testing sets to be populated with the same metrics or there is no apples-apples comparison



<hr style="height:2px;border-width:0;color:black;background-color:black">

# Part 2 Cleanup of Alive vs Dead

After cleaning up the dataset as a whole, we should dive deeper and cleanup based on the two main types of patients: Alive vs Dead which end up being training and testing sets.  We need a common set of features between both of these groups or we wont have an Apples to Apples comparison

<mark>The dataset contains 3 Patient Groups (Disposition ID): Died in Hospice, Died in Home Health Care, Alive so we now need to move to the 2nd stage of EDA and that is getting a clean deadpatients list</mark>



<div class="alert alert-info" role="alert">
    <b>% of null values by Disposition ID</b>
</div>

# Missing Values (Pct Total) for Disposition 3 Patients (Hospice)

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==3].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==3])).sort_values(ascending=False)
mask3 = (df.loc[:,colsOASIS][df['DispositionId']==3].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==3])>=.75).sort_values(ascending=False)
#df.loc[:, ~mask]



In [ ]:
df.loc[:,colsOASIS].loc[:, mask3].columns
mask3_sr = df.loc[:,colsOASIS].loc[:, mask3].columns

# Missing Values (Pct Total) for Disposition 2 Patients (HHG)

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])).sort_values(ascending=False)
mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

In [ ]:
df.loc[:,colsOASIS].loc[:, mask2].columns
mask2_sr = df.loc[:,colsOASIS].loc[:, mask2].columns

# Factors Missing All Values for All Dead Patients (Disposition 2 and 3 combined) - We can Eliminate from df

In [ ]:
#(mask3_sr).difference(mask2_sr)
#(mask2_sr).difference(mask3_sr)
#mask2_sr.union(mask3_sr)
#mask2_sr.intersection(mask3_sr)
len(mask2_sr.union(mask3_sr))
mask2_sr.union(mask3_sr)

____

# There are no fields completley null for the alive patients cohort except there ae some close

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])).sort_values(ascending=False)
mask1 = (df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])>=.75).sort_values(ascending=False)
#mask1




In [ ]:
#bigmask = (mask2_sr.intersection(mask3_sr)).intersection(df.loc[:,colsOASIS].loc[:, mask1].columns)
#bigmask = (mask2_sr.union(mask3_sr)).intersection(df.loc[:,colsOASIS].loc[:, mask1].columns)
bigmask = (mask2_sr.union(mask3_sr)).union(df.loc[:,colsOASIS].loc[:, mask1].columns)
bigmask

In [ ]:
#(1-(df[(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df[(df['DispositionId']==2) + (df['DispositionId']==3)])).sort_values(ascending=False)
(df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum()/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)])).sort_values(ascending=False)


# Depending on the formula we can wither see True = Column is above threshold, or True if Column is below

In [ ]:
fullalive = (1-(df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1]))>=.90).sort_values(ascending=False)
emptyalive = (df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])>=.90).sort_values(ascending=False)
emptyalive
maskfilter1 = emptyalive

In [ ]:
fulldead = ((1-(df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)]))>=.90).sort_values(ascending=False)
emptydead = (((df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)]))>=.90).sort_values(ascending=False)
emptydead
maskfilter23 = emptydead #((1-(df[(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df[(df['DispositionId']==2) + (df['DispositionId']==3)]))==1).sort_values(ascending=False)

In [ ]:
len(df.loc[:,colsOASIS].loc[:, maskfilter1].columns)
(df.loc[:,colsOASIS].loc[:, maskfilter1].columns)
len(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)
(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)

In [ ]:
len((df.loc[:,colsOASIS].loc[:, maskfilter1].columns).intersection(df.loc[:,colsOASIS].loc[:, maskfilter23].columns))
(df.loc[:,colsOASIS].loc[:, maskfilter1].columns).intersection(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)

<div class="alert alert-danger" role="alert">
    <b>Dropping Columns that are NULL for Dead Patients</b>
</div>

<div class="alert alert-danger" role="alert">
    <b>Dropping Columns with too many null values across Dead Patients (Dispo 2 and Dispo 3)</b>
</div>

In [ ]:
finaldrop = (df.loc[:,colsOASIS].loc[:, maskfilter23].columns)[:-1].union(bigmask)
len(finaldrop)


In [ ]:
df = df.drop(finaldrop, axis=1)

In [ ]:
len(df.columns)

<div class="alert alert-block alert-success">
    <b>Export Cleaned DataSet</b>
</div>

In [ ]:
df.to_csv((csvStore+'/cleaned_data.csv'), sep=',', encoding='utf-8', index=False)

<div class="alert alert-block alert-success">
    <b>EDF for our final Factor List</b>
</div>

In [ ]:
#analyzing the dataset
high_level_cleaned = sv.analyze(df, pairwise_analysis = 'off')
#display the report
high_level_cleaned .show_html('high_level_cleaned.html')

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-warning" role="alert">
    <b>Tools to Filter DF and understand the dataset</b>
</div>


<div class="alert alert-warning" role="alert">
    <b>Enumerate the columns to help to more quickly ID them</b>
</div>


In [ ]:
column_list = pd.DataFrame(enumerate(df.columns.values.tolist())).set_index(0)
column_list

In [ ]:
df.dtypes

<div class="alert alert-warning" role="alert">
    <b> Filter Tool to find contents of specific cells</b>
</div>

In [ ]:
df.iloc[13,107] # add rows if i want to find a specific cell
df.iloc[13:14,:10] # add rows if i want to find a specific cell

<div class="alert alert-warning" role="alert">
    <b>Eliminate Columns with only 1 distinct value</b>
</div>


In [ ]:
cols=[]
for col in df.columns:
    if len(df[col].unique()) >= 1 and len(df[col].unique()) <= 3:
        cols.append(col)
len(cols)

In [ ]:
cols

In [ ]:
df.nunique().sort_values(ascending=False)

The number of unique values in a column provide insight into the nature of the column.<br>
The OASIS data is standardized so there are only a few responses the nurses must choose.<br>
pa_id will have many more unique value, followed by episode id, all the way to a primary key with a unique value for each row<br>
<br>
We Eliminated Columns From Dead Patients if all the values were missing.<br>
There is no Point in keeping these columns for the independent variables either

<hr style="height:2px;border-width:0;color:black;background-color:black">

# Part 2.1 Remove Duplicate Rows

# START HERE w Base Case dataset

In [ ]:
df = pd.read_csv(csvStore+'/cleaned_data.csv', dtype = {'ceas_ascode': object, 'M1322_NBR_PRSULC_STG1':object})

In [ ]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

In [ ]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OfEpisode', case=False)  )] 
df[cols_w_dates] = df[cols_w_dates].apply(pd.to_datetime)



cols_w_dates = cols_w_dates.union(df.columns[(df.columns.str.contains('DAYS', case=False))])

cols_w_dates = ['epi_SocDate', 'epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT', 'M0090_INFO_COMPLETED_DT', 'cedd_dateofdeath', 'DaysToDeath']


firstcols = [ 'DispositionId', 'pa_id', 'year_born', 'pa_gender', 'epi_id', 'ceo_id']
first_cols = firstcols+cols_w_dates

move_columns = first_cols + (df.columns.drop(first_cols).tolist())
df = df[move_columns]



<div class="alert alert-danger">
  <strong>Create a GPU version of DFF for faster processing</strong> 
</div>

In [ ]:
dff = cudf.DataFrame(df)
len(dff)

<div class="alert alert-success" role="alert">
    <b>Create Filters that remove the primary keys</b>
</div>

In [ ]:
dffcols3 = dff.columns.difference(['Unnamed: 0', 'ceo_HIPPS', 'ceo_HHRG', 'ceo_id'])
df_deduped3 = dff.drop_duplicates(subset=dffcols3)
df_deduped3.shape # This is the same because it has keep by default


<div class="alert alert-info" role="alert">
    <b>This makes sure that when removing duplicates we keep the rows that have HIPPS codes</b>
</div>

In [ ]:
uniq_indx = (dff.sort_values(by="ceo_HIPPS", na_position='last').drop_duplicates(subset=dffcols3, keep='first')).index

In [ ]:
uniq_indx = dff.loc[uniq_indx]
len(uniq_indx)

In [ ]:
dtale.show(uniq_indx.to_pandas().head(100))

<div class="alert alert-block alert-success">
    <b>Export De-Duped Dataset</b>
</div>

In [ ]:
uniq_indx.to_pandas().to_csv(csvStore+'/deduped_complete.csv', sep=',', encoding='utf-8', index=False)
dff = uniq_indx
df = dff.to_pandas()

In [ ]:
len(df)
len(dff)
type(df)

# Part 3 - Get Patient Lists

In [ ]:
dff.groupby('DispositionId').agg({'pa_id': 'nunique'})

# This group of patients has more than 1 Disposition ID which was discovered when counting the number of unique patient id's in the entire dataset vs the number of unique patients with each disposition

In [ ]:
len(dff.pa_id.unique())
grp = dff.groupby('pa_id').nunique()['DispositionId'].apply(lambda g: g>1)
len(grp[grp].index)
grp[grp].index
dff[dff['pa_id'].isin(grp[grp].index)].reset_index(drop=True).to_csv(csvStore+'/du.csv', index=False)
len(dff)

In [ ]:
dtale.show(dff[dff['pa_id'].isin(grp[grp].index)].to_pandas().reset_index(drop=True).sort_values(by=['pa_id', 'DispositionId']))

In [ ]:
df_Disposition = pd.read_csv(csvStore+'/du.csv', dtype = {'ceas_ascode': object, 'M1322_NBR_PRSULC_STG1':object})

In [ ]:
if mixed_dtypes := {c: dtype for c in df_Disposition.columns if (dtype := pd.api.types.infer_dtype(df_Disposition[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

In [ ]:
    
cols_w_dates = df_Disposition.columns[(df_Disposition.columns.str.contains('DT', case=False) | df_Disposition.columns.str.contains('date', case=False) | df_Disposition.columns.str.contains('OfEpisode', case=False)  )] 
df_Disposition[cols_w_dates] = df_Disposition[cols_w_dates].apply(pd.to_datetime)

In [ ]:
dtale.show(df_Disposition.head())

In [ ]:
query = '''SELECT pa_id, min(epi_SocDate) as SOC, max(epi_DischargeDate) as final_dschg
           FROM df_Disposition 
           GROUP BY pa_id
           HAVING (JULIANDAY(final_dschg) - JULIANDAY(SOC) < 365)
            '''
deaddupes = sqldf(query)
deaddupeset = set(deaddupes.pa_id)


querya = '''SELECT pa_id, min(epi_SocDate) as SOC, max(epi_DischargeDate) as final_dschg
           FROM df_Disposition 
           GROUP BY pa_id
           HAVING (JULIANDAY(final_dschg) - JULIANDAY(SOC) > 365)
            '''

            
alivedupes = sqldf(querya)
alivedupeset = set(alivedupes.pa_id)

dtale.show(df_Disposition[df_Disposition.pa_id.isin(deaddupeset)])

# Remove Patients with Duplicate Dispositions in name of cleanliness

In [ ]:
dff = dff[~(dff['pa_id'].isin(grp[grp].index) & (dff.DispositionId == 1)) ]
dtale.show(dff.to_pandas())

## Eliminate rows and Patients where the SOC date doesn't equal the 1st episode

In [ ]:
df = dff.to_pandas()

In [ ]:


df_Patient_list1 = set(df.pa_id[((df.DispositionId == 1) & (df['epi_SocDate'] == df['epi_StartOfEpisode']))])#.to_list())
#df_Patient_list1 = set(df.pa_id[(((df.DispositionId == 1) | (df.DispositionId == 2)) & (df['epi_SocDate'] == df['epi_StartOfEpisode']))])#.to_list())
df_Patient_list2 = set(df.pa_id[((df.DispositionId == 1) & (df['epi_SocDate'] != df['epi_StartOfEpisode']))])#.to_list())
#df_Patient_list2 = set(df.pa_id[(((df.DispositionId == 1) | (df.DispositionId == 2)) & (df['epi_SocDate'] != df['epi_StartOfEpisode']))])#.to_list())
df_Patient_list3 = set(df.pa_id[((df.DispositionId == 1) & (df['epi_StartOfEpisode'] <= (df['epi_SocDate']) + pd.Timedelta(120, unit='D')  ))])
df_Patient_list4 = set(df.pa_id[(((df.DispositionId == 2) | (df.DispositionId == 3)) & (df['epi_SocDate'] == df['epi_StartOfEpisode']))])#.to_list())
df_Patient_list5 = set(df.pa_id[(((df.DispositionId == 2) | (df.DispositionId == 3)) & (df['epi_SocDate'] != df['epi_StartOfEpisode']))])#.to_list())
df_Patient_list6 = set(df.pa_id[((df.DispositionId == 3) & (df['epi_SocDate'] == df['epi_StartOfEpisode']))])#.to_list())
df_Patient_list7 = set(df.pa_id[((df.DispositionId == 3) & (df['epi_SocDate'] != df['epi_StartOfEpisode']))])#.to_list())




f"There are {len(df_Patient_list1)} alive patients whose SOC date matches their first episode date"
f"There are {len(df_Patient_list2)} alive patients whose SOC date DOES NOT MATCH their first episode date"


f"There are {len(df_Patient_list3)} alive patients whose SOC date is 120 days earlier than their first episode"
f"There are {len(df_Patient_list4)} dead HHG patients whose SOC date matches their first episode date"


f"There are {len(df_Patient_list5)} dead HHG patients whose SOC date matches their first episode date"
f"There are {len(df_Patient_list6)} dead hospice patients whose SOC date matches their first episode date"
f"There are {len(df_Patient_list7)} dead hospice patients whose SOC date DOES NOT MATCH their first episode date"



len(df_Patient_list1.intersection(df_Patient_list2))
len(df_Patient_list3.intersection(df_Patient_list4))
len(df_Patient_list5.intersection(df_Patient_list6))

df_SOC = (pd.DataFrame(df_Patient_list6))


df_SOC.to_csv(csvStore+'/soc2.csv', sep=',', encoding='utf-8', index=False)


> Within each disposition their are a minority of patients who have SOC dates, earlier than their first episode.  Only within the alive patients do we have individual patients with both date types

<div class="alert alert-danger" role="alert">
    <b>Get our list of Dead Patients</b>
</div>

<div class="alert alert-block alert-info">
    <b>Hospice Patients with cedd_dataofdeath within 1 year from Start of Care</b>
</div>


In [ ]:
dtale.show(df[df.DispositionId == 3].head(15))
dead_hospice_patients = set(df.pa_id[(df.DispositionId == 3) & (df.DaysToDeath < 366)].to_list())
dead_366 = set(df.pa_id[(df.DispositionId == 3) & (df.DaysToDeath >= 366)].to_list())
f"There are {len(dead_hospice_patients)}, patients who died in hospice care less than a year after transferring and {len(dead_hospice_patients)+ len(dead_366)} in total"

<div class="alert alert-block alert-info">
    <b>These 3 Patients Had Days to Death Greater Than 365</b>
</div>



In [ ]:
hospice366 = set(df.pa_id[(df.DispositionId == 3) & (df.DaysToDeath > 365)].to_list())
dtale.show(df[['pa_id', 'DaysToDeath', 'pa_gender']][df.pa_id.isin(hospice366)])

<div class="alert alert-block alert-info">
    <b>Home Health Patients who Died at home less than 1 year after Starting Care</b>
</div>



In [ ]:
dtale.show(df[(df.DispositionId == 2) & (df.M0100_ASSMT_REASON == '8 - DEATH AT HOME')].head(15))
HHP_DeadAtHome = set(df.pa_id[(df.DispositionId == 2) & (df.dcc_desc == 'PATIENT EXPIRED') & (df.dcr_desc == 'PATIENT EXPIRED') & (df.M0100_ASSMT_REASON == '8 - DEATH AT HOME') & (df.M0906_DC_TRAN_DTH_DT <= (df.epi_SocDate + pd.Timedelta(365, unit='D')))].to_list())
f"There are {len(HHP_DeadAtHome)}, HH patients who died at home within 1 year of SOC"

In [ ]:
HHP_Dead = set(df.pa_id[(df.DispositionId == 2) & (df.dcc_desc == 'PATIENT EXPIRED') & (df.dcr_desc == 'PATIENT EXPIRED') & (df.M0100_ASSMT_REASON == '8 - DEATH AT HOME')].to_list())
len(HHP_Dead)

# There are 22 patients who died at home but took longer than 1 year

In [ ]:
len(HHP_Dead.intersection(HHP_DeadAtHome))
len(HHP_Dead.difference(HHP_DeadAtHome))# Makes sense because one is subset of the other: 484 died at home period, 462 died at home with the time criteria 

In [ ]:
HHP_Dead.difference(HHP_DeadAtHome)

### Home Health Care Patients Labeled with both dcr_dsec and dcr_desc Expired but no indication of Death via nurse evaluation

In [ ]:
dtale.show(df[(df.DispositionId == 2) & (df.M0100_ASSMT_REASON != '8 - DEATH AT HOME')].head(15))
HHP_expired = set(df.pa_id[(df.DispositionId == 2) & (df.dcc_desc == 'PATIENT EXPIRED') & (df.dcr_desc == 'PATIENT EXPIRED') & (df.M0100_ASSMT_REASON != '8 - DEATH AT HOME') & (df.M0906_DC_TRAN_DTH_DT <= (df.epi_SocDate + pd.Timedelta(365, unit='D')))].to_list())
f"There are {len(HHP_expired)}, patients who have rows with no death at home"

In [ ]:
HHP_expired2 = set(df.pa_id[(((df.DispositionId == 2) | (df.DispositionId == 1)) & (df.dcc_desc == 'PATIENT EXPIRED') & (df.dcr_desc == 'PATIENT EXPIRED') & (df.epi_DischargeDate >= (df.epi_SocDate + pd.Timedelta(365, unit='D'))))].to_list())
HHP_expired3 = set(df.pa_id[((df.DispositionId == 1) & (df.epi_DischargeDate >= (df.epi_SocDate + pd.Timedelta(365, unit='D'))))].to_list())

f"There are {len(HHP_expired2)}, patients who have rows with no death at home"
f"There are {len(HHP_expired3)}, patients who have rows with no death at home"

In [ ]:
#Out of the 821 patients who have assessments with no indication of heath at home, 754 of them never did at all but 67 did.
#This means that 67 patients meet both criteria: in other words they have a death at home note, but also additional rows
# The 754 have no indication of a death at home

len(HHP_DeadAtHome.intersection(HHP_expired))
len(HHP_expired.intersection(HHP_DeadAtHome))
len(HHP_expired.union(HHP_DeadAtHome))

len(HHP_expired.difference(HHP_DeadAtHome)) # 754 + 67 = 821 patients who didnt
len(HHP_DeadAtHome.difference(HHP_expired)) # 395 + 67 = 462 HH patients who are confirmed died at home in less than 1 year
#sett = HHP_DeadAtHome.difference(HHP_expired)
#sett = HHP_expired.difference(HHP_DeadAtHome)
sett = HHP_expired.intersection(HHP_DeadAtHome)



died_at_home = HHP_DeadAtHome.difference(HHP_expired)



In [ ]:
len(sett)

In [ ]:
dtale.show(df[['pa_id', 'DispositionId', 'DaysToDeath', 'pa_gender', 'dcc_desc', 'dcr_desc', 'M0906_DC_TRAN_DTH_DT', 'epi_SocDate' ]][df.pa_id.isin(sett)])

## Get combined Set of Dead Patients

In [ ]:
deadlistset = dead_hospice_patients.union(died_at_home)
deadlist = pd.DataFrame(deadlistset)
f"There are {len(deadlist)}, patients who are reliably confirmed dead within 1 year of SOC, either in hospice, or at home, "

In [ ]:
df_dead = dff[dff['pa_id'].isin(deadlistset)]
df_dead.info()
dtale.show(df_dead.to_pandas())

In [ ]:
len((dff[dff.DispositionId == 1]))

In [ ]:
dff.groupby('DispositionId').agg({'pa_id': 'nunique'})
df_dead.groupby('DispositionId').agg({'pa_id': 'nunique'})

<hr style="height:2px;border-width:0;color:black;background-color:black">

# Newer Approach

In [ ]:
qHospiceDead = '''SELECT pa_id, dcc_desc, dcr_desc, DispositionId, DaysToDeath 
           FROM df 
           WHERE DaysToDeath < 365'''

qHHDead = '''SELECT pa_id, dcc_desc, dcr_desc, DispositionId, DaysToDeath, JULIANDAY(epi_DischargeDate) -  JULIANDAY(epi_SocDate) as INT
           FROM df 
           WHERE DispositionID != 3 AND (dcc_desc = 'PATIENT EXPIRED' and dcr_desc = 'PATIENT EXPIRED' and INT < 365)'''



hospiceDead = sqldf(qHospiceDead)
HHDead = sqldf(qHHDead)


In [ ]:
hosDeadset = set(hospiceDead.pa_id)
HHDeadset = set(HHDead.pa_id)
deadlistset = hosDeadset.union(HHDeadset).union(deaddupeset)

df_dead = df[df.pa_id.isin(deadlistset)]
f"There are {len(deadlistset)}, patients who died in less than 1 year"

In [ ]:

dtale.show(df_dead)

<div class="alert alert-success" role="alert">
    <b>Export Dataset with Bad Columns Removed and Dead Patients with more than 365 days removed</b>
</div>

In [ ]:
df_dead.to_csv(csvStore+'/dead_patients_final.csv', sep=',', encoding='utf-8', index=False)

# Get Patients who have lived longer than 1 year

In [ ]:
HHP_dead.intersection(HHP_alive)
len(HHP_dead.intersection(HHP_alive))


dtale.show(df[df.pa_id.isin(HHP_dead.intersection(HHP_alive))])

In [ ]:
#HHP_alive = set(df.pa_id[((df.epi_DischargeDate) >= ((df.epi_SocDate | df.cedd_dateofdeath) + pd.Timedelta(365, unit='D')) ) ].to_list())

query = '''SELECT pa_id, JULIANDAY(cedd_dateofdeath) -  JULIANDAY(epi_SocDate) as INT,  JULIANDAY(epi_DischargeDate) -  JULIANDAY(epi_SocDate) as INT2 
           FROM df 
           WHERE INT > 365 or INT2 > 365 or DaysToDeath > 365'''

alive = sqldf(query)

HHP_alive = set(alive.pa_id).union(alivedupeset)
HHP_alive = set(HHP_alive)

df_alive = df[df.pa_id.isin(HHP_alive)]

f"There are only {len(HHP_alive)}, patients who have lived longer than 1 year"

In [ ]:
#dtale.show(df[((df.DispositionId == 1) & (df.epi_DischargeDate >= (df.epi_SocDate + pd.Timedelta(365, unit='D'))))])
dtale.show(df[df.pa_id.isin(HHP_alive)])

<div class="alert alert-block alert-success">
    <b>Export Alive List</b>
</div>

In [ ]:
alivelist = pd.DataFrame(HHP_alive)
df_alive.to_csv(csvStore+'/alive_patients_final.csv', sep=',', encoding='utf-8', index=False)
f"There are {len(alivelist)}, patients in the dataset who lived longer than 1 year"

In [ ]:
neither = deadlistset.union(HHP_alive)    
conditions = [df['pa_id'].isin(HHP_alive), df['pa_id'].isin(deadlistset), (~df['pa_id'].isin(neither))]
values = ['0', '1', None]
df.insert(2, "mortality", np.select(conditions, values))                                                           

In [ ]:
dtale.show(df.head(1000))

In [ ]:
df.groupby('mortality').agg({'pa_id': 'nunique'})

In [ ]:
dtale.show(df[~df['mortality'].isnull()])


<div class="alert alert-danger" role="alert">
    <b>Remove Weird Rows where Patient number differ for hospice transfers</b>
</div>

In [ ]:
df.groupby('pa_id').agg({'ceo_id': 'nunique'}).sort_values('ceo_id', ascending=False)

In [ ]:
(df.groupby('ceo_id').agg({'pa_id': 'nunique'}).sort_values('pa_id', ascending=False))

In [ ]:
df_visits = (df.groupby('ceo_id').agg({'pa_id': 'nunique'}).sort_values('pa_id', ascending=False).pa_id==2).reset_index()
visits = df_visits.ceo_id[df_visits.pa_id==True]

In [ ]:
visits

In [ ]:
len(df[~((df.ceo_id.isin(visits)) & (df.DispositionId != 3))])
df = df[~((df.ceo_id.isin(visits)) & (df.DispositionId != 3))]

<div class="alert alert-info" role="alert">
    <b>Final Data Sets</b>
</div>

In [ ]:
df_final = df[~df['mortality'].isnull()]
df_eliminated = df[df['mortality'].isnull()]
df_final.to_csv(csvStore+'/final.csv', sep=',', encoding='utf-8', index=False)
df_eliminated.to_csv(csvStore+'/eliminated.csv', sep=',', encoding='utf-8', index=False)